<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/intro-Machine-Learning/blob/main/classes/class_16_end_to_end_1/class_16_end_to_end_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table> 

## [Video de apoyo a la lectura de esta clase](https://www.youtube.com/watch?v=4JDqj1NT4Qk)

Pasos para creación del cuaderno interactivo.

1. Plantear bien la pregunta.
   * ¿Regresión o clasificación?
   * ¿Tipo de regresión y tipo de clasificación?

2. Exploración inicial.
   * Indicar la fuente de dónde se toman los datos.
   * Hacer explícita la función objetivo.
   * Decir cuáles son los atributos (descripción breve de cada uno)
   * Practicar una primera exploración gráfica de los datos.

3. Preparar los datos para los algoritmos de aprendizaje.
   * Hacer separación inicial de datos para entrenar y para testear.
   * Limpiar datos y llenar datos faltantes.
   * Explorar correlaciones lineales con la variable objetivo.
   * Eliminar de ser necesario atributos que no sean de mucha utilidad.
   * Ingeniería de atributos.
   * Estandarizar los datos.
   * Crear funciones en Python de manera que se puedan replicar los procesos de transformación de datos en proyectos nuevos.

4. Entrenamiento y selección de modelo.
   * Instanciar varios modelos y entrenarlos sobre datos de entrenamiento preparados.
   * Medir el desempeño de varios modelos (comparativa, con la técnica de la validación cruzada).

5. Afinar el modelo.
   * Crear cuadrícula (de búsqueda) de hiperparámetros.
   * Seleccionar la combinación de hiperparámetros que consigue el mejor puntaje. (El mejor modelo).

6. Presentar la solución.
   * Mostrar el desempeño sobre los datos para testear.
   * (Opcional) Gráfico intuitivo para representar el modelo.

7. (Opcional) ¿Cómo se sacará provecho del modelo en otros proyectos?

[Video de apoyo sobre presentación de la metodología para modelo de machine Learning](https://www.youtube.com/watch?v=blRXFU2KooI)

# Implementación del plan 

1. Plantear bien la pregunta.
   * ¿Regresión o clasificación?
   * ¿Tipo de regresión y tipo de clasificación?

Tenemos un dataset con la siguiente configuración: $[X|y]$.  

$X = [x_{ij}] \in \mathbb{R}_{n,m}$.  
$X^{j}$: $j$ ésimo atributo.   
$X_{i}$: $i$ ésima fila o instancia ($i$ -ésimo distrito)  
$x_{ij}$: $ij$ ésima entrada de la matriz $X$. 
$y = [y_{i}] \in \mathbb{R}^{n}$: el vector de los valores promedios de vivienda.  
$y_{i}$ el valor promedio de vivienda en el $i$ ésimo distrito

Este es un problema de regresión porque lo que se trata es de predecir un valor o la función predictora de valor continuo o de valores en un intervalo de números reales. 

El tipo de regresión es lineal porque 

$$h(X_{i}) = w \cdot X_{i} = w_{0} + w_{1}x_{i1} + \cdots + w_{n}x_{im} = \begin{pmatrix} w_{0} \\ w_{1} \\ \vdots \\ w_{m} \end{pmatrix} \cdot \begin{pmatrix} 1 \\ x_{i1} \\ \cdots \\ x_{im} \end{pmatrix} $$

donde $m$ es el número de atributos. 



# 2. Exploración inicial.

##  Indicar la fuente de dónde se toman los datos.

Su primera tarea es utilizar los datos del censo de California para construir un modelo de precios de viviendas en el estado.

Estos datos incluyen métricas como la población, el ingreso medio y el precio medio de la vivienda para cada grupo de bloques en California.

Los grupos de bloques son la unidad geográfica más pequeña para la que la Oficina del Censo de EE. UU. publica datos de muestra (un grupo de bloques suele tener una población de 600 a 3000 personas).

Los llamaremos “distritos” para abreviar.

Su modelo debe aprender de estos datos y poder predecir el precio medio de la vivienda en cualquier distrito, dadas todas las demás métricas.

# Hacer explícita la función objetivo.

$$ h:\mathbb{R}^{9} \to \mathbb{R} $$

$$ h(X_{n \times m}) = y $$

donde $X$ es una matriz alta (número de filas nucho mayor al número de columnas). 

$y$ es un vector de $\mathbb{R}^{m}$ cuyas entradas son los valores promedio de vivienda por distrito. 

$$ h(X_{i}) = y_{i} $$

## Decir cuáles son los atributos (descripción breve de cada uno)

Son nueve atributos o variables predictoras entre las que están:

* longitud
* latitud
* habitaciones
* dormitorios
* ingresos promedio
* proximidad al oceano
* antiguedad promedio de las viviendas en el distrito.
* Número de hogares
* población


## [Video de apoyo]()

## Practicar una primera exploración gráfica de los datos.

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 


In [ ]:
v = pd.read_csv('vivienda.csv')  

In [ ]:
v.head() 

In [ ]:
v.info()

In [ ]:
v.proximidad.value_counts() 

In [ ]:
v.describe() 

In [ ]:
import matplotlib.pyplot as plt

v.hist(figsize = (20,10))

plt.show() 

## Visualización de datos geográficos 


In [ ]:
v.plot(kind = 'scatter', x = 'longitud', y = 'latitud', alpha = 1)

plt.show() 

In [ ]:
v.plot(kind = 'scatter', x = 'longitud', y = 'latitud', alpha = 0.1) 

plt.show() 

In [ ]:
v.plot(kind = 'scatter', x = 'longitud', y = 'latitud', alpha = 0.4, \
      s = v.población/100, label = 'Población', \
      c = 'valor', cmap = plt.get_cmap('jet'), colorbar = True, figsize = (12, 8))  

# s de size o tamaño del punto. 

plt.show() 

# 3. Preparar los datos para los algoritmos de aprendizaje.

## Hacer separación inicial de datos para entrenar y para testear.

In [ ]:
import numpy as np 
np.random.seed(42)

def dividir_entrenamiento_testeo(datos, porcentaje_testeo):
    indices_reordenados = np.random.permutation(len(datos))
    tamaño_conjunto_testeo = int(len(datos)*porcentaje_testeo)
    indices_entrenamiento = indices_reordenados[tamaño_conjunto_testeo:]
    indices_testeo = indices_reordenados[:tamaño_conjunto_testeo]
    return datos.iloc[indices_entrenamiento], datos.iloc[indices_testeo]

In [ ]:
v_train, v_test = dividir_entrenamiento_testeo(v, 0.2)

In [ ]:
len(v_train), len(v_test) 

In [ ]:
len(v_train)/len(v_test) 


In [ ]:
from sklearn.model_selection import train_test_split

v_train, v_test = train_test_split(v, test_size = 0.2, random_state = 42)

## Limpiar datos y llenar datos faltantes.

In [ ]:
mediana  = v.dormitorios.mean() 

In [ ]:
mediana 

In [ ]:
v.dormitorios.fillna(mediana)

In [ ]:
len(pd.DataFrame(v.dormitorios.fillna(mediana)))

## Explorar correlaciones lineales con la variable objetivo.

   * Eliminar de ser necesario atributos que no sean de mucha utilidad.
   * Ingeniería de atributos.
   * Estandarizar los datos.
   * Crear funciones en Python de manera que se puedan replicar los procesos de transformación de datos en proyectos nuevos.

In [ ]:
def dividir_train_test(datos, porcentaje_testeo):
    import numpy as np 
    indices_reordenados = np.random.permutation(len(v))
    return indices_reordenados[:5]

In [ ]:
dividir_train_test(v,0.2) 

In [ ]:
def dividir_train_test(datos, porcentaje_testeo):
    import numpy as np 
    indices_reordenados = np.random.permutation(len(v))
    tamaño_entrenamiento = int(len(v)*(1-porcentaje_testeo))
    train_set = v.iloc[indices_reordenados[:tamaño_entrenamiento]]
    test_set = v.iloc[indices_reordenados[tamaño_entrenamiento:]]
    return train_set, test_set

In [ ]:
v_train, v_test = dividir_train_test(v,0.2) 

In [ ]:
v_train.head() 

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
v_train, v_test = train_test_split(v,test_size = 0.2,random_state = 42)

## Establecer correlaciones 

<img src = ''>

In [ ]:
matriz_correlación = v.corr() 

In [ ]:
matriz_correlación

In [ ]:
matriz_correlación.valor.sort_values(ascending = False) 

In [ ]:
from pandas.plotting import scatter_matrix

atributos = ['valor', 'ingresos', 'salas', 'antiguedad','longitud', 'latitud'] 

scatter_matrix(v[atributos], figsize = (20,10)) 
plt.show() 

In [ ]:
v.plot(kind = 'scatter', x = 'ingresos', y = 'valor')  

# Experimentación con combinación de atributos 

Con suerte, las secciones anteriores le dieron una idea de algunas formas en que puede explorar los datos y obtener información.

Identificó algunas peculiaridades de los datos que quizás desee limpiar antes de enviar los datos a un algoritmo de aprendizaje automático y encontró correlaciones interesantes entre los atributos, en particular con el atributo de destino.

También notó que algunos atributos tienen una distribución de cola pesada, por lo que es posible que desee transformarlos (por ejemplo, calculando su logaritmo).

Por supuesto, su millaje variará considerablemente con cada proyecto,
pero las ideas generales son similares

Una última cosa que quizás desee hacer antes de preparar los datos para los algoritmos de Machine Learning es probar varias combinaciones de atributos.

Por ejemplo, el número total de habitaciones en un distrito no es muy útil si no sabe cuántos hogares hay. Lo que realmente desea es el número de habitaciones por hogar. Del mismo modo, el número total de dormitorios por sí solo no es muy útil:
probablemente quieras compararlo con el número de habitaciones.

Y la población por hogar también parece una combinación de atributos interesante para observar.

Vamos a crear estos nuevos atributos:

In [ ]:
v["salas_por_hogar"] = v["salas"]/v["hogares"]

In [ ]:
v["habitaciones_por_sala"] = v["habitaciones"]/v["salas"]

In [ ]:
v["población_por_hogar"]=v["población"]/v["hogares"] 

And now let’s look at the correlation matrix again:

In [ ]:
corr_matrix = v.corr()
corr_matrix['valor'].sort_values(ascending=False)

## Tiempos 

1. Clase 17: 
   * Construir en dataframe y sistetizar la información y visualizarla 
   * división en entrenamiento y testeo.
   * Establecer corralaciones y definir nuevos atributos.

2. Calse 18
   * imputar los datos faltantes
   * escalar los datos
   * entrenar el modelo lineal
   * medir su desempeño. 